In [ ]:
import openpyxl
import pandas as pd
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import basedosdados as bd
import numpy as np


In [ ]:
## Acessando pelo pacote da BD

df=bd.read_sql("SELECT  * FROM `basedosdados-projetos.republica.indicadores_estadic_perfil_gestor` where tema='educacao'", billing_project_id="repositoriodedadosgpsp", from_file=False, reauth=False, use_bqstorage_api=False)

In [ ]:
df.head()

In [ ]:
## mudando para escolaridade do gestor pois gestores costumam ter escolaridade acima de ensino superior, o que não faria sentido dentro das faixas que usamos em outros contextos. 
df = df.rename(columns={"escolaridade":"escolaridade_gestor"})

In [ ]:
## agrupar faixa etária 


df['faixa_etária']=np.where((df['idade']>=18) & (df['idade']<30),"Entre 18 e 29 anos",0)
df['faixa_etária']=np.where((df['faixa_etária'] !=0) & (df['idade']>=30) & (df['idade']<50),"Entre 30 e 49 anos",df['faixa_etária'])
df['faixa_etária']=np.where((df['faixa_etária'] !=0) & (df['idade']>=50) & (df['idade']<65),"Entre 50 e 64 anos",df['faixa_etária'])
df['faixa_etária']=np.where((df['faixa_etária'] !=0) & (df['idade']>=65),"Acima de 65 anos",df['faixa_etária'])


In [ ]:
df['faixa_etária'].unique()

In [ ]:
df.columns

In [ ]:
interesse = ['ano', 'sigla_uf', 'tema', 'caracterizacao_orgao_gestor', 'sexo',
       'idade','faixa_etária', 'cor_raca', 'cor_raca_autodeclarado', 'escolaridade_gestor',
       'escolaridade_formacao']

In [ ]:
df1 = df[interesse]

In [ ]:
df1.head()

In [ ]:
### Subindo para o GBQ

In [ ]:
client = bigquery.Client()

In [ ]:
dataset_ref = client.dataset('cargos_lideranca')

In [ ]:
df1 = df1.rename(columns={"faixa_etária":"faixa_etaria"})

In [85]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 0 to 26
Data columns (total 11 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ano                          27 non-null     Int64 
 1   sigla_uf                     27 non-null     object
 2   tema                         27 non-null     object
 3   caracterizacao_orgao_gestor  27 non-null     object
 4   genero                       27 non-null     object
 5   idade                        27 non-null     Int64 
 6   faixa_etaria                 27 non-null     object
 7   cor_raca                     27 non-null     object
 8   cor_raca_autodeclarado       27 non-null     Int64 
 9   escolaridade_gestor          27 non-null     object
 10  escolaridade_formacao        26 non-null     object
dtypes: Int64(3), object(8)
memory usage: 3.7+ KB


In [ ]:
df1.columns

In [86]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
 bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação'),
 bigquery.SchemaField('tema','STRING',description='Tema de origem daquele dado na ESTADIC'),
 bigquery.SchemaField('caracterizacao_orgao_gestor','STRING',description='Tipo de estrutura do órgão responsável por aquele tema investigado. Neste caso, qual estrutura de estado comporta o tema de educação.'),
 bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
 bigquery.SchemaField('idade','INTEGER',description='idade da observação'),
 bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação'),
 bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
 bigquery.SchemaField('cor_raca_autodeclarado','INTEGER',description='Se a raça/cor foi autodeclarada ou não. Assume-se 1 se autodeclarou e 0 se não autodeclarou.'),
 bigquery.SchemaField('escolaridade_gestor','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
 bigquery.SchemaField('escolaridade_formacao','STRING',description='Área deformação.')
 ]

In [87]:
table_ref = dataset_ref.table('ESTADIC_perfil_gestor_educacao')

In [88]:
job_config = bigquery.LoadJobConfig(schema=schema)

In [89]:
job = client.load_table_from_dataframe(df1,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=4554e30b-e9fb-4382-87d2-d630fa63639e>